In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_2c03c86718a14172be72f4b49bdc6575 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='36DkZMq17QsN1gK_XmbSdL9SCJgt1ovAjnemTp80gCq3',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_2c03c86718a14172be72f4b49bdc6575.get_object(Bucket='surveyanalysis-donotdelete-pr-8av9vykjkhhkfs',Key='siteldata.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,feedback
0,Great service great customer service warranty ...
1,I'm in my late 60's and have always had Michel...
2,It took quite a few phone calls but once I got...
3,Agent on the phone call was very professional ...
4,Michelin lives up to their warrentees


In [4]:
!pip install ibm_watson

     |████████████████████████████████| 378kB 8.9MB/s eta 0:00:01
     |████████████████████████████████| 204kB 31.0MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/71/9a/0a/9b3ca8eca69bc5362eb04709a750b30055a9d27818fd0c9494
  Stored in directory: /home/dsxuser/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson ibm-cloud-sdk-core


In [9]:
import pandas as pd
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions,SentimentOptions
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
authenticator = IAMAuthenticator('bwCwitpwww1TqWSPx60rM-FI87BopdOApVloAO8VGR36')

# Authentication via external config like VCAP_SERVICES
service = NaturalLanguageUnderstandingV1(
    version='2018-03-16', authenticator=authenticator)
# service.set_disable_ssl_verification(True)
service.set_service_url('https://gateway.watsonplatform.net/natural-language-understanding/api')

In [10]:
df1 = df_data_1.head(10)
df1

,feedback
0,Great service great customer service warranty ...
1,I'm in my late 60's and have always had Michel...
2,It took quite a few phone calls but once I got...
3,Agent on the phone call was very professional ...
4,Michelin lives up to their warrentees
5,I have been very happy with your response so q...
6,My tires came with the new car I purchased fro...
7,I bought a Toyota Rav4 that had Michelin tires...
8,I thought when I bought a new vehicle that the...
9,I like a company that stands behind its produc...


In [11]:
def sentiment_and_keyword(st,service=service):
    return (service.analyze(text=st,features=Features(keywords=KeywordsOptions(sentiment= True,emotion= True,limit= 3),sentiment=SentimentOptions())).get_result())

In [12]:
df1['sentiment_keyword_json'] = df1.apply(lambda row : sentiment_and_keyword(str(row['feedback'])),axis = 1)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
df2 = df1[['feedback','sentiment_keyword_json']]

In [14]:
def sentiment_extractor(row_num):
    sentiment_scores=df2['sentiment_keyword_json'][row_num]['sentiment']['document']
    return sentiment_scores

def keyword_extractor(row_num):
    keyword_scores=[{df2['sentiment_keyword_json'][row_num]['keywords'][i]['text'] : df2['sentiment_keyword_json'][row_num]['keywords'][i]['sentiment']['score']} for i in range(0,len(df2['sentiment_keyword_json'][row_num]['keywords']))]
    return keyword_scores

df2['sentiment_score'] = df2.apply(lambda row : sentiment_extractor(row.name),axis = 1)
df2['keyword_score'] = df2.apply(lambda row : keyword_extractor(row.name),axis = 1)
df2

,feedback,sentiment_keyword_json,sentiment_score,keyword_score
0,Great service great customer service warranty ...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.999466, 'label': 'positive'}","[{'Great service': 0.999466}, {'great customer..."
1,I'm in my late 60's and have always had Michel...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.580746, 'label': 'positive'}","[{'Michelin tires': -0.384194}, {'family membe..."
2,It took quite a few phone calls but once I got...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': -0.862452, 'label': 'negative'}","[{'phone calls': -0.866019}, {'complaint numbe..."
3,Agent on the phone call was very professional ...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.972038, 'label': 'positive'}","[{'phone call': 0.972038}, {'Agent': 0.972038}]"
4,Michelin lives up to their warrentees,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0, 'label': 'neutral'}","[{'Michelin': 0}, {'warrentees': 0}]"
5,I have been very happy with your response so q...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.963326, 'label': 'positive'}","[{'Michelin tires': 0.959063}, {'fact': 0.9560..."
6,My tires came with the new car I purchased fro...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': -0.332891, 'label': 'negative'}","[{'next service': -0.861172}, {'prior Michelin..."
7,I bought a Toyota Rav4 that had Michelin tires...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': -0.77172, 'label': 'negative'}","[{'Toyota Rav4': 0}, {'Michelin tires': 0}, {'..."
8,I thought when I bought a new vehicle that the...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': -0.932289, 'label': 'negative'}","[{'new vehicle': -0.914493}, {'tires': -0.9144..."
9,I like a company that stands behind its produc...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.595588, 'label': 'positive'}","[{'customer service': -0.554345}, {'mileage': ..."
